In [11]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# CONFIGURATION
DATA_PATH = "C:/Users/kehin/Downloads/mimic-iii-extracted/"

# Compliance Language Lexicon
LEXICON = {
    'negative_compliance': [
        'non-compliant', 'noncompliant', 'non compliant',
        'uncooperative', 'un-cooperative', 'resistant',
        'refuses', 'refused', 'refusing', 'refusal',
        'difficult', 'combative', 'aggressive', 'hostile',
        'agitated', 'demanding', 'manipulative', 
        'drug-seeking', 'drug seeking', 'narcotic-seeking',
        'non-adherent', 'nonadherent', 'non adherent'
    ],
    'positive_compliance': [
        'compliant', 'cooperative', 'pleasant',
        'agreeable', 'appropriate', 'follows instructions',
        'adherent', 'complies', 'willing', 'cooperative'
    ],
    'neutral_descriptors': [
        'alert', 'oriented', 'responsive',
        'stable', 'calm', 'resting', 'comfortable'
    ]
}


print("MIMIC-III CLINICAL TEXT BIAS ANALYSIS")
print("Focus: Racial Bias in Compliance Language")
print()


MIMIC-III CLINICAL TEXT BIAS ANALYSIS
Focus: Racial Bias in Compliance Language



## Load datasets

In [21]:
# Load Dataset
print("Step 1: Loading MIMIC-III data files...")
print("-"*70)

# Load the three main files
patients = pd.read_csv(r"C:\Users\kehin\Downloads\mimic-iii-extracted\mimic-iii-clinical-database-1.4\PATIENTS.csv.gz")
admissions = pd.read_csv(r"C:\Users\kehin\Downloads\mimic-iii-extracted\mimic-iii-clinical-database-1.4\ADMISSIONS.csv.gz")
notes = pd.read_csv(r"C:\Users\kehin\Downloads\mimic-iii-extracted\mimic-iii-clinical-database-1.4\NOTEEVENTS.csv.gz")
print(f"✓ PATIENTS.csv: {len(patients):,} patients")
print(f"✓ ADMISSIONS.csv: {len(admissions):,} admissions")
print(f"✓ NOTEEVENTS.csv: {len(notes):,} clinical notes")
print()


Step 1: Loading MIMIC-III data files...
----------------------------------------------------------------------
✓ PATIENTS.csv: 46,520 patients
✓ ADMISSIONS.csv: 58,976 admissions
✓ NOTEEVENTS.csv: 2,083,180 clinical notes



In [25]:
# STEP 3: Merging demographic data
print("Step 3: Merging demographic data...")
print("-" * 70)

# Filter notes to only include discharge summaries
discharge_notes = notes[notes['CATEGORY'] == 'Discharge summary'].copy()

# Merge discharge notes with admissions data
merged = discharge_notes.merge(
    admissions[['SUBJECT_ID', 'HADM_ID', 'ETHNICITY', 'ADMISSION_TYPE', 'ADMITTIME']], 
    on=['SUBJECT_ID', 'HADM_ID'],
    how='left'
)

print(f"Merged dataset created with {len(merged):,} rows")


Step 3: Merging demographic data...
----------------------------------------------------------------------
Merged dataset created with 59,652 rows


In [31]:
# Start fresh - reload discharge notes
discharge_notes = notes[notes['CATEGORY'] == 'Discharge summary'].copy()

print(f"Starting with {len(discharge_notes)} discharge notes")
print()

# Step 1: Merge with admissions
print("Step 1: Merging with admissions...")
merged = discharge_notes.merge(
    admissions[['SUBJECT_ID', 'HADM_ID', 'ETHNICITY', 'ADMISSION_TYPE', 'ADMITTIME']], 
    on=['SUBJECT_ID', 'HADM_ID'],
    how='left'
)
print(f"After admissions merge: {len(merged)} rows")
print(f"Columns: {merged.columns.tolist()}")
print()

# Step 2: Check patients data before merge
print("Step 2: Checking patients data...")
print(f"Patients columns: {patients.columns.tolist()}")
print(f"First few patient SUBJECT_IDs: {patients['SUBJECT_ID'].head()}")
print(f"First few merged SUBJECT_IDs: {merged['SUBJECT_ID'].head()}")
print()

# Step 3: Try the patients merge
print("Step 3: Merging with patients...")
merged = merged.merge(
    patients[['SUBJECT_ID', 'GENDER', 'DOB']], 
    on='SUBJECT_ID',
    how='left'
)
print(f"After patients merge: {len(merged)} rows")
print(f"Columns: {merged.columns.tolist()}")
print()

# Check if DOB is now present
if 'DOB' in merged.columns:
    print("✓ DOB column successfully merged!")
    print(f"Non-null DOB values: {merged['DOB'].notna().sum()}")
else:
    print("✗ DOB column NOT in merged data")
    print("This means the merge failed - investigating why...")

Starting with 59652 discharge notes

Step 1: Merging with admissions...
After admissions merge: 59652 rows
Columns: ['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CHARTTIME', 'STORETIME', 'CATEGORY', 'DESCRIPTION', 'CGID', 'ISERROR', 'TEXT', 'ETHNICITY', 'ADMISSION_TYPE', 'ADMITTIME']

Step 2: Checking patients data...
Patients columns: ['ROW_ID', 'SUBJECT_ID', 'GENDER', 'DOB', 'DOD', 'DOD_HOSP', 'DOD_SSN', 'EXPIRE_FLAG']
First few patient SUBJECT_IDs: 0    249
1    250
2    251
3    252
4    253
Name: SUBJECT_ID, dtype: int64
First few merged SUBJECT_IDs: 0    22532
1    13702
2    13702
3    13702
4    26880
Name: SUBJECT_ID, dtype: int64

Step 3: Merging with patients...
After patients merge: 59652 rows
Columns: ['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CHARTTIME', 'STORETIME', 'CATEGORY', 'DESCRIPTION', 'CGID', 'ISERROR', 'TEXT', 'ETHNICITY', 'ADMISSION_TYPE', 'ADMITTIME', 'GENDER', 'DOB']

✓ DOB column successfully merged!
Non-null DOB values: 59652


In [33]:
# LEXICON-BASED ANALYSIS
print("Step 4: Applying compliance language lexicon...")
print("-"*70)

def count_lexicon_terms(text, term_list):
    """Count occurrences of terms from a list in text"""
    if pd.isna(text):
        return 0
    text_lower = str(text).lower()
    count = 0
    for term in term_list:
        # Use word boundaries to avoid partial matches
        pattern = r'\b' + re.escape(term) + r'\b'
        count += len(re.findall(pattern, text_lower))
    return count

# Apply lexicon counting
print("Counting negative compliance terms...")
merged['NEGATIVE_COMPLIANCE'] = merged['TEXT'].apply(
    lambda x: count_lexicon_terms(x, LEXICON['negative_compliance'])
)

print("Counting positive compliance terms...")
merged['POSITIVE_COMPLIANCE'] = merged['TEXT'].apply(
    lambda x: count_lexicon_terms(x, LEXICON['positive_compliance'])
)

print("Counting neutral descriptor terms...")
merged['NEUTRAL_TERMS'] = merged['TEXT'].apply(
    lambda x: count_lexicon_terms(x, LEXICON['neutral_descriptors'])
)

# Calculate text length and term density
merged['TEXT_LENGTH'] = merged['TEXT'].apply(lambda x: len(str(x).split()))
merged['NEGATIVE_DENSITY'] = (merged['NEGATIVE_COMPLIANCE'] / merged['TEXT_LENGTH'] * 1000).fillna(0)
merged['POSITIVE_DENSITY'] = (merged['POSITIVE_COMPLIANCE'] / merged['TEXT_LENGTH'] * 1000).fillna(0)

print(f" Lexicon analysis complete")
print()


Step 4: Applying compliance language lexicon...
----------------------------------------------------------------------
Counting negative compliance terms...
Counting positive compliance terms...
Counting neutral descriptor terms...
 Lexicon analysis complete



In [36]:
# DETECT UNIQUE PATTERNS
print("Detecting unique patterns")


def detect_patterns(text):
    """Detect specific patterns in clinical text"""
    if pd.isna(text):
        return []
    text_lower = str(text).lower()
    patterns = []
    
    # Drug-seeking behavior
    if re.search(r'\bdrug.{0,10}seeking\b', text_lower):
        patterns.append('drug-seeking')
    
    # Left AMA (Against Medical Advice)
    if 'ama' in text_lower or 'against medical advice' in text_lower:
        patterns.append('AMA')
    
    # Frequent ED visits
    if re.search(r'\bfrequent\s+(ed|emergency)', text_lower):
        patterns.append('frequent-ED')
    
    # Non-compliance mentioned
    if 'non-compliant' in text_lower or 'noncompliant' in text_lower:
        patterns.append('non-compliant-documented')
    
    return patterns

merged['PATTERNS'] = merged['TEXT'].apply(detect_patterns)
merged['HAS_PATTERNS'] = merged['PATTERNS'].apply(lambda x: len(x) > 0)

print(f" Pattern detection complete")
print()

Detecting unique patterns
 Pattern detection complete



In [42]:
# Check what columns exist in merged
print("Current columns in merged dataframe:")
print(merged.columns.tolist())
print()

# Check if ETHNICITY column exists
if 'ETHNICITY' in merged.columns:
    print(" ETHNICITY column exists")
    print("\nFirst few ethnicity values:")
    print(merged['ETHNICITY'].head(10))
    print()
    
    # Now create RACE_SIMPLIFIED
    def simplify_ethnicity(eth):
        if pd.isna(eth):
            return 'UNKNOWN'
        eth = str(eth).upper()
        if 'WHITE' in eth:
            return 'WHITE'
        elif 'BLACK' in eth or 'AFRICAN AMERICAN' in eth:
            return 'BLACK'
        elif 'HISPANIC' in eth or 'LATINO' in eth:
            return 'HISPANIC'
        elif 'ASIAN' in eth:
            return 'ASIAN'
        elif 'UNKNOWN' in eth or 'UNABLE' in eth or 'DECLINED' in eth:
            return 'UNKNOWN'
        else:
            return 'OTHER'
    
    merged['RACE_SIMPLIFIED'] = merged['ETHNICITY'].apply(simplify_ethnicity)
    
    print("✓ RACE_SIMPLIFIED column created")
    print("\nRace distribution:")
    print(merged['RACE_SIMPLIFIED'].value_counts())
    print()
else:
    print("✗ ETHNICITY column doesn't exist")
    print("Need to go back and fix the merge with admissions")

Current columns in merged dataframe:
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CHARTTIME', 'STORETIME', 'CATEGORY', 'DESCRIPTION', 'CGID', 'ISERROR', 'TEXT', 'ETHNICITY', 'ADMISSION_TYPE', 'ADMITTIME', 'GENDER', 'DOB', 'NEGATIVE_COMPLIANCE', 'POSITIVE_COMPLIANCE', 'NEUTRAL_TERMS', 'TEXT_LENGTH', 'NEGATIVE_DENSITY', 'POSITIVE_DENSITY', 'PATTERNS', 'HAS_PATTERNS']

✓ ETHNICITY column exists

First few ethnicity values:
0    UNKNOWN/NOT SPECIFIED
1                    WHITE
2                    WHITE
3                    WHITE
4                    WHITE
5                    WHITE
6                    WHITE
7                    WHITE
8       HISPANIC OR LATINO
9       HISPANIC OR LATINO
Name: ETHNICITY, dtype: object

✓ RACE_SIMPLIFIED column created

Race distribution:
RACE_SIMPLIFIED
WHITE       42255
UNKNOWN      6409
BLACK        5721
HISPANIC     2034
OTHER        1717
ASIAN        1516
Name: count, dtype: int64



In [46]:
# Now compute the race statistics
print("Computing statistics by race")

race_stats = merged.groupby('RACE_SIMPLIFIED').agg({
    'SUBJECT_ID': 'count',
    'NEGATIVE_COMPLIANCE': ['sum', 'mean', 'std'],
    'POSITIVE_COMPLIANCE': ['sum', 'mean', 'std'],
    'NEUTRAL_TERMS': ['mean'],
    'NEGATIVE_DENSITY': ['mean', 'std'],
    'POSITIVE_DENSITY': ['mean', 'std'],
    'HAS_PATTERNS': 'sum'
}).round(3)

race_stats.columns = ['_'.join(col).strip() for col in race_stats.columns.values]
race_stats = race_stats.rename(columns={'SUBJECT_ID_count': 'NOTE_COUNT'})

# Calculate negative rate percentage
race_stats['NEGATIVE_RATE_%'] = (
    race_stats['NEGATIVE_COMPLIANCE_sum'] / 
    (race_stats['NEGATIVE_COMPLIANCE_sum'] + race_stats['POSITIVE_COMPLIANCE_sum']) * 100
).round(2)

print("Statistics computed")
print()
print("Summary Statistics by Race:")
print(race_stats[['NOTE_COUNT', 'NEGATIVE_COMPLIANCE_mean', 
                   'POSITIVE_COMPLIANCE_mean', 'NEGATIVE_RATE_%']])
print()

# Show some key findings
print("KEY FINDINGS:")
for race in race_stats.index:
    neg_mean = race_stats.loc[race, 'NEGATIVE_COMPLIANCE_mean']
    pos_mean = race_stats.loc[race, 'POSITIVE_COMPLIANCE_mean']
    neg_rate = race_stats.loc[race, 'NEGATIVE_RATE_%']
    count = race_stats.loc[race, 'NOTE_COUNT']
    
    print(f"{race}:")
    print(f"  • Notes: {count:,}")
    print(f"  • Avg negative terms per note: {neg_mean:.2f}")
    print(f"  • Avg positive terms per note: {pos_mean:.2f}")
    print(f"  • Negative rate: {neg_rate:.1f}%")
    print()

Computing statistics by race
Statistics computed

Summary Statistics by Race:
                 NOTE_COUNT  NEGATIVE_COMPLIANCE_mean  \
RACE_SIMPLIFIED                                         
ASIAN                  1516                     0.507   
BLACK                  5721                     0.782   
HISPANIC               2034                     0.550   
OTHER                  1717                     0.580   
UNKNOWN                6409                     0.507   
WHITE                 42255                     0.604   

                 POSITIVE_COMPLIANCE_mean  NEGATIVE_RATE_%  
RACE_SIMPLIFIED                                             
ASIAN                               0.554            47.79  
BLACK                               0.592            56.93  
HISPANIC                            0.590            48.23  
OTHER                               0.525            52.50  
UNKNOWN                             0.358            58.60  
WHITE                               0.

In [53]:
# Calculate AGE more carefully to handle MIMIC-III's date anonymization
print("Calculating age (handling MIMIC-III date anonymization)...")

# Convert to datetime
merged['DOB'] = pd.to_datetime(merged['DOB'], errors='coerce')
merged['ADMITTIME'] = pd.to_datetime(merged['ADMITTIME'], errors='coerce')

# Calculate age safely
def calculate_age_safe(row):
    try:
        if pd.isna(row['DOB']) or pd.isna(row['ADMITTIME']):
            return np.nan
        
        age = (row['ADMITTIME'] - row['DOB']).days / 365.25
        
        # MIMIC-III shifts dates for patients >89 to year 2200+
        # If age is negative or > 150, it's anonymized
        if age < 0 or age > 150:
            return 90  # Use 90 as proxy for elderly patients
        elif age > 89:
            return 90  # Cap at 90 as per MIMIC-III convention
        else:
            return age
    except:
        return np.nan

merged['AGE'] = merged.apply(calculate_age_safe, axis=1)

# Create age groups
merged['AGE_GROUP'] = pd.cut(
    merged['AGE'], 
    bins=[0, 18, 35, 50, 65, 100],
    labels=['Under 18', '18-34', '35-49', '50-64', '65+'],
    include_lowest=True
)

print(" Age calculated")
print(f"\nAge statistics:")
print(merged['AGE'].describe())
print(f"\nAge distribution:")
print(merged['AGE_GROUP'].value_counts().sort_index())
print()

# Now export with all columns
print("Exporting spreadsheets...")
print("="*70)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Export 1: Detailed Records
detailed_export = merged[[
    'SUBJECT_ID', 'HADM_ID', 'AGE', 'AGE_GROUP', 'GENDER',
    'ETHNICITY', 'RACE_SIMPLIFIED', 'ADMISSION_TYPE',
    'TEXT_LENGTH', 'NEGATIVE_COMPLIANCE', 'POSITIVE_COMPLIANCE',
    'NEUTRAL_TERMS', 'NEGATIVE_DENSITY', 'POSITIVE_DENSITY',
    'PATTERNS', 'HAS_PATTERNS'
]].copy()

detailed_export['PATTERNS'] = detailed_export['PATTERNS'].apply(lambda x: ', '.join(x) if x else '')
detailed_file = f"mimic_detailed_records_{timestamp}.csv"
detailed_export.to_csv(detailed_file, index=False)
print(f" Detailed records: {detailed_file}")

# Export 2: Race Statistics
stats_file = f"mimic_race_statistics_{timestamp}.csv"
race_stats.to_csv(stats_file)
print(f" Race statistics: {stats_file}")

# Export 3: Lexicon Reference
lexicon_df = pd.DataFrame([
    {'Category': 'Negative Compliance', 'Term Count': 19, 
     'Examples': 'non-compliant, noncompliant, resistant, refuses, uncooperative, difficult, combative, aggressive, hostile, drug-seeking'},
    {'Category': 'Positive Compliance', 'Term Count': 10, 
     'Examples': 'compliant, cooperative, pleasant, agreeable, appropriate'},
    {'Category': 'Neutral Descriptors', 'Term Count': 7, 
     'Examples': 'alert, oriented, responsive, stable, calm'}
])
lexicon_file = f"mimic_lexicon_reference_{timestamp}.csv"
lexicon_df.to_csv(lexicon_file, index=False)
print(f"Lexicon reference: {lexicon_file}")

# Export 4: Cross-tabulation (Race x Admission Type)
crosstab = pd.crosstab(
    merged['RACE_SIMPLIFIED'],
    merged['ADMISSION_TYPE'],
    values=merged['NEGATIVE_COMPLIANCE'],
    aggfunc='mean'
).round(2)
crosstab_file = f"mimic_race_admissiontype_crosstab_{timestamp}.csv"
crosstab.to_csv(crosstab_file)
print(f" Cross-tabulation: {crosstab_file}")

print()
print(" ALL FILES EXPORTED SUCCESSFULLY!")
print(f"\nTotal notes analyzed: {len(merged):,}")
print(f"Notes with negative compliance terms: {(merged['NEGATIVE_COMPLIANCE'] > 0).sum():,}")
print(f"Notes with positive compliance terms: {(merged['POSITIVE_COMPLIANCE'] > 0).sum():,}")
print(f"Notes with unique patterns: {merged['HAS_PATTERNS'].sum():,}")
print()
print("Files created:")
print(f"  1. {detailed_file}")
print(f"  2. {stats_file}")
print(f"  3. {lexicon_file}")
print(f"  4. {crosstab_file}")
print()

Calculating age (handling MIMIC-III date anonymization)...
 Age calculated

Age statistics:
count    56797.000000
mean        58.152594
std         22.845068
min          0.000000
25%         48.361396
50%         62.904860
75%         75.279945
max         88.999316
Name: AGE, dtype: float64

Age distribution:
AGE_GROUP
Under 18     4346
18-34        3551
35-49        7525
50-64       15398
65+         25977
Name: count, dtype: int64

Exporting spreadsheets...
 Detailed records: mimic_detailed_records_20251103_101407.csv
 Race statistics: mimic_race_statistics_20251103_101407.csv
Lexicon reference: mimic_lexicon_reference_20251103_101407.csv
 Cross-tabulation: mimic_race_admissiontype_crosstab_20251103_101407.csv

 ALL FILES EXPORTED SUCCESSFULLY!

Total notes analyzed: 59,652
Notes with negative compliance terms: 19,273
Notes with positive compliance terms: 19,939
Notes with unique patterns: 9,259

Files created:
  1. mimic_detailed_records_20251103_101407.csv
  2. mimic_race_statist

In [55]:
# Display the race statistics in detail
print("RACIAL BIAS ANALYSIS - KEY FINDINGS")
print()
print(race_stats)
print()

# Calculate some interesting comparisons
print("COMPARATIVE ANALYSIS:")

# Compare BLACK vs WHITE
if 'BLACK' in race_stats.index and 'WHITE' in race_stats.index:
    black_neg = race_stats.loc['BLACK', 'NEGATIVE_COMPLIANCE_mean']
    white_neg = race_stats.loc['WHITE', 'NEGATIVE_COMPLIANCE_mean']
    diff_pct = ((black_neg - white_neg) / white_neg * 100)
    
    print(f"\nNegative Compliance Language:")
    print(f"  • BLACK patients: {black_neg:.3f} terms/note")
    print(f"  • WHITE patients: {white_neg:.3f} terms/note")
    print(f"  • Difference: {diff_pct:+.1f}%")
    
    black_pos = race_stats.loc['BLACK', 'POSITIVE_COMPLIANCE_mean']
    white_pos = race_stats.loc['WHITE', 'POSITIVE_COMPLIANCE_mean']
    diff_pos_pct = ((black_pos - white_pos) / white_pos * 100)
    
    print(f"\nPositive Compliance Language:")
    print(f"  • BLACK patients: {black_pos:.3f} terms/note")
    print(f"  • WHITE patients: {white_pos:.3f} terms/note")
    print(f"  • Difference: {diff_pos_pct:+.1f}%")

# Show all races ranked by negative compliance
print(f"\n\nRaces Ranked by Negative Compliance (highest to lowest):")
ranked = race_stats.sort_values('NEGATIVE_COMPLIANCE_mean', ascending=False)
for race in ranked.index:
    neg_mean = ranked.loc[race, 'NEGATIVE_COMPLIANCE_mean']
    count = ranked.loc[race, 'NOTE_COUNT']
    print(f"  {race:12s}: {neg_mean:.3f} terms/note  (n={count:,})")

RACIAL BIAS ANALYSIS - KEY FINDINGS

                 NOTE_COUNT  NEGATIVE_COMPLIANCE_sum  \
RACE_SIMPLIFIED                                        
ASIAN                  1516                      769   
BLACK                  5721                     4475   
HISPANIC               2034                     1118   
OTHER                  1717                      996   
UNKNOWN                6409                     3251   
WHITE                 42255                    25537   

                 NEGATIVE_COMPLIANCE_mean  NEGATIVE_COMPLIANCE_std  \
RACE_SIMPLIFIED                                                      
ASIAN                               0.507                    1.070   
BLACK                               0.782                    1.338   
HISPANIC                            0.550                    1.114   
OTHER                               0.580                    1.174   
UNKNOWN                             0.507                    1.071   
WHITE                   

In [59]:
from scipy import stats

print("STATISTICAL SIGNIFICANCE TESTS")

# Chi-square test: Are negative compliance rates different by race?
# Create contingency table
contingency = pd.crosstab(
    merged['RACE_SIMPLIFIED'],
    merged['NEGATIVE_COMPLIANCE'] > 0
)

chi2, p_value, dof, expected = stats.chi2_contingency(contingency)
print(f"\nChi-Square Test (Negative Compliance by Race):")
print(f"  χ² = {chi2:.2f}, p-value = {p_value:.2e}")
print(f"  Result: {'SIGNIFICANT' if p_value < 0.001 else 'Not significant'} (p < 0.001)")

# T-test: BLACK vs WHITE negative compliance
black_notes = merged[merged['RACE_SIMPLIFIED'] == 'BLACK']['NEGATIVE_COMPLIANCE']
white_notes = merged[merged['RACE_SIMPLIFIED'] == 'WHITE']['NEGATIVE_COMPLIANCE']

t_stat, p_value_ttest = stats.ttest_ind(black_notes, white_notes)
print(f"\nT-Test (BLACK vs WHITE - Negative Compliance):")
print(f"  t = {t_stat:.3f}, p-value = {p_value_ttest:.2e}")
print(f"  Result: {'SIGNIFICANT' if p_value_ttest < 0.001 else 'Not significant'} (p < 0.001)")

# Effect size (Cohen's d)
cohens_d = (black_notes.mean() - white_notes.mean()) / np.sqrt(
    ((len(black_notes)-1)*black_notes.std()**2 + (len(white_notes)-1)*white_notes.std()**2) / 
    (len(black_notes) + len(white_notes) - 2)
)
print(f"  Cohen's d = {cohens_d:.3f} ({'small' if abs(cohens_d) < 0.5 else 'medium' if abs(cohens_d) < 0.8 else 'large'} effect)")

print("\n")

STATISTICAL SIGNIFICANCE TESTS

Chi-Square Test (Negative Compliance by Race):
  χ² = 235.08, p-value = 8.70e-49
  Result: SIGNIFICANT (p < 0.001)

T-Test (BLACK vs WHITE - Negative Compliance):
  t = 10.320, p-value = 6.07e-25
  Result: SIGNIFICANT (p < 0.001)
  Cohen's d = 0.145 (small effect)




In [67]:
import pandas as pd
from datetime import datetime

print(" Exporting all results to one Excel workbook...")

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
excel_file = f"mimic_analysis_results_{timestamp}.xlsx"

# Prepare all dataframes
detailed_export = merged[[
    'SUBJECT_ID', 'HADM_ID', 'AGE', 'AGE_GROUP', 'GENDER',
    'ETHNICITY', 'RACE_SIMPLIFIED', 'ADMISSION_TYPE',
    'TEXT_LENGTH', 'NEGATIVE_COMPLIANCE', 'POSITIVE_COMPLIANCE',
    'NEUTRAL_TERMS', 'NEGATIVE_DENSITY', 'POSITIVE_DENSITY',
    'PATTERNS', 'HAS_PATTERNS'
]].copy()

# Convert list patterns to readable text
detailed_export['PATTERNS'] = detailed_export['PATTERNS'].apply(lambda x: ', '.join(x) if isinstance(x, list) else '')

lexicon_df = pd.DataFrame([
    {'Category': 'Negative Compliance', 'Term Count': len(LEXICON['negative_compliance']), 
     'Examples': ', '.join(LEXICON['negative_compliance'][:10])},
    {'Category': 'Positive Compliance', 'Term Count': len(LEXICON['positive_compliance']), 
     'Examples': ', '.join(LEXICON['positive_compliance'][:10])},
    {'Category': 'Neutral Descriptors', 'Term Count': len(LEXICON['neutral_descriptors']), 
     'Examples': ', '.join(LEXICON['neutral_descriptors'][:10])}
])

crosstab = pd.crosstab(
    merged['RACE_SIMPLIFIED'],
    merged['ADMISSION_TYPE'],
    values=merged['NEGATIVE_COMPLIANCE'],
    aggfunc='mean'
).round(2)

# Save everything to one Excel workbook
with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
    detailed_export.to_excel(writer, sheet_name='Detailed Records', index=False)
    race_stats.to_excel(writer, sheet_name='Race Statistics')
    lexicon_df.to_excel(writer, sheet_name='Lexicon Reference', index=False)
    crosstab.to_excel(writer, sheet_name='Race x Admission Type')

print(f" All results exported to: {excel_file}")

print()
print("ANALYSIS COMPLETE!")
print("Next Steps:")
print(" Open the Excel file and explore the tabs.")
print(" Review race statistics for disparities.")
print(" Examine cross-tabulations for patterns.")
print(" Consider running statistical significance tests.")


 Exporting all results to one Excel workbook...
 All results exported to: mimic_analysis_results_20251103_114041.xlsx

ANALYSIS COMPLETE!
Next Steps:
 Open the Excel file and explore the tabs.
 Review race statistics for disparities.
 Examine cross-tabulations for patterns.
 Consider running statistical significance tests.


In [71]:
import pandas as pd
from datetime import datetime

# Assume 'merged' is your DataFrame containing all your analysis
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

excel_file_path = rf"C:\Users\kehin\word-bias\scripts\mimic_analysis_{timestamp}.xlsx"

# Export the full DataFrame to Excel
merged.to_excel(excel_file_path, index=False, sheet_name='Analysis')

print(" Excel file created at:", excel_file_path)


 Excel file created at: C:\Users\kehin\word-bias\scripts\mimic_analysis_20251103_115048.xlsx


In [77]:
!pip install XlsxWriter
import pandas as pd
from datetime import datetime

# Timestamp for file naming
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
excel_file_path = rf"C:\Users\kehin\word-bias\scripts\mimic_analysis_{timestamp}.xlsx"

#  Sheet 1: Detailed Records 
detailed_export = merged.copy()
# If PATTERNS column is a list, join them as string
if 'PATTERNS' in detailed_export.columns:
    detailed_export['PATTERNS'] = detailed_export['PATTERNS'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

# Sheet 2: Lexicon Reference 
lexicon_df = pd.DataFrame([
    {'Category': 'Negative Compliance', 'Term Count': len(LEXICON['negative_compliance']),
     'Examples': ', '.join(LEXICON['negative_compliance'][:10])},
    {'Category': 'Positive Compliance', 'Term Count': len(LEXICON['positive_compliance']),
     'Examples': ', '.join(LEXICON['positive_compliance'][:10])},
    {'Category': 'Neutral Descriptors', 'Term Count': len(LEXICON['neutral_descriptors']),
     'Examples': ', '.join(LEXICON['neutral_descriptors'][:10])}
])

# Optional Cross-tab (Race x Admission Type) 
crosstab = pd.crosstab(
    merged['RACE_SIMPLIFIED'],
    merged['ADMISSION_TYPE'],
    values=merged['NEGATIVE_COMPLIANCE'],
    aggfunc='mean'
).round(2)

# Export to Excel with multiple sheets
with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    detailed_export.to_excel(writer, sheet_name='Detailed_Records', index=False)
    lexicon_df.to_excel(writer, sheet_name='Lexicons', index=False)
    crosstab.to_excel(writer, sheet_name='Race_Admission_Crosstab')

print(" Excel workbook created successfully at:", excel_file_path)


 Excel workbook created successfully at: C:\Users\kehin\word-bias\scripts\mimic_analysis_20251103_120831.xlsx


In [93]:
from collections import Counter

# Initialize a Counter
unknown_counter = Counter()

# Iterate row by row to avoid memory issues
for terms in merged['UNKNOWN_TERMS']:
    unknown_counter.update(terms)

# Convert to DataFrame
unknown_df = pd.DataFrame({
    'Unknown_Term': list(unknown_counter.keys()),
    'Frequency': list(unknown_counter.values())
})

print(f"Total unique unknown terms: {len(unknown_df)}")


Total unique unknown terms: 984662
